In [ ]:
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas_datareader import data
from dask.distributed import Client
from dask.distributed import as_completed
import openpyxl
import xlsxwriter

In [ ]:
end = datetime.today()
start = end - relativedelta(years=2)

In [ ]:
end.strftime('%Y-%m-%d')

In [ ]:
SP500ticker_list = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
SP600ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1]
SP400ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0]


In [ ]:
sp500 = SP500ticker_list['Symbol']
sp600 = SP600ticker_list['Ticker symbol']
sp400 = SP400ticker_list['Ticker symbol']


In [ ]:
len(sp500)+len(sp600)+len(sp400)

In [ ]:
SP1500 = list()
SP1500.extend(sp500)
sp600 = list(set(sp600).difference(SP1500))
print(len(sp600))
SP1500.extend(sp600)
sp400 = list(set(sp400).difference(SP1500))
print(len(sp400))
SP1500.extend(set(sp400).difference(SP1500))

In [ ]:
sp1500_index_df = pd.DataFrame()

lists = ["sp500","sp600","sp400"]

for l in lists:
    if(l=='sp500'):
        df = pd.DataFrame(sp500,columns=['Symbol'])
        print(len(df))
        #range_ = 
        #print(range_)
        df['Market Index'] = pd.Series(["SP500" for x in range(0,len(df.index),1)])
    elif(l=='sp600'):
        df = pd.DataFrame(np.array(sp600),columns=['Symbol'])        
        print(len(df))        
        df['Market Index'] = pd.Series(["SP600" for x in range(len(df.index))])
        df.index = range(len(sp500),(len(sp500)+len(df.index)),1)        
    else:
        df = pd.DataFrame(np.array(sp400),columns=['Symbol'])
        print(len(df))
        df['Market Index'] = pd.Series(["SP400" for x in range(len(df.index))])
        df.index = range((len(sp500)+len(sp600)),(len(sp500)+len(sp600)+len(df.index)),1)
    sp1500_index_df = pd.concat([sp1500_index_df,df],axis=0)

In [ ]:
len(sp1500_index_df)-len(SP1500)

In [ ]:
def getStock(npa):
    #position = npa[0]
    symbol = npa[0]
    start_=npa[1]
    end_=npa[2]
    #print(symbol)
    
    #tickerdata.history(period="2Y")
    #tickerdata = yf.Ticker(symbol)
    #data_ = tickerdata.history(start=start_,end=end_)
    data_ = yf.download(symbol, start=start_,end=end_)
    #data_['Sector'] = tickerdata.info['sector']
    #data_['Symbol'] = symbol
    #data_['Market Cap'] = data.get_quote_yahoo(names[n])['marketCap']
    #data_['Sector'] = tickerdata.info['sector']
    
    """
    tickerdata.info
    tickerdata.quarterly_balance_sheet
    tickerdata.quarterly_cashflow
    tickerdata.quarterly_dividends
    tickerdata.quarterly_earnings
    tickerdata.quarterly_financials
    tickerdata.actions
    tickerdata.mutualfund_holders
    tickerdata.options
    tickerdata.sustainability
    """
    
    return([symbol,data_])

In [ ]:
def getMarketCap(npa):
    symbol = npa[0]

    #start_=npa[2]
    #end_=npa[3]
    #print(symbol)
    
    #tickerdata.history(period="2Y")
    #tickerdata = yf.Ticker(symbol)
    #data_ = tickerdata.history(start=start_,end=end_)
    data_ = data.get_quote_yahoo(symbol)['marketCap']
    #data_['Sector'] = tickerdata.info['sector']
    
    """
    tickerdata.info
    tickerdata.quarterly_balance_sheet
    tickerdata.quarterly_cashflow
    tickerdata.quarterly_dividends
    tickerdata.quarterly_earnings
    tickerdata.quarterly_financials
    tickerdata.actions
    tickerdata.mutualfund_holders
    tickerdata.options
    tickerdata.sustainability
    """
    
    return([symbol,data_])

In [ ]:
def getSector(npa):
    #position = npa[0]
    symbol = npa[0]
    #start_=npa[2]
    #end_=npa[3]
    
    tickerdata = yf.Ticker(symbol)
    return([symbol,tickerdata.info['sector']])

In [ ]:
def getFinObject(npa):
    #position = npa[0]
    symbol = npa[0]
    #start_=npa[2]
    #end_=npa[3]
    
    tickerdata = yf.Ticker(symbol)
    return([symbol,tickerdata])

In [167]:
def calculateMinMax(data_):
    data_df = data_
    n_ = data_df['Symbol'].values[-1]
    
    latest_p = pd.DataFrame(data_df)['Close'][-1]
    oneYearDate = pd.DataFrame(data_df)['Close'][[-1]].index[0] - relativedelta(years=1)
    
    past_year = data_df[data_df.index>=oneYearDate.strftime('%Y-%m-%d')]
    
    #np.max(npa[np.where(np.array(names_prices)=='GBX')[0][0]]['High'])
    max_2y = np.max(pd.DataFrame(data_df)['High'])
    min_2y = np.min(pd.DataFrame(data_df)['Low'])
    quantiles_close_2y = pd.DataFrame(data_df)['Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    std_2y = np.std(pd.DataFrame(data_df)['Close'])
    mean_2y = np.mean(pd.DataFrame(data_df)['Close'])
    
    max_1y = np.max(past_year['High'])
    min_1y = np.min(past_year['Low'])
    quantiles_close_1y = pd.DataFrame(data_df)['Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    std_1y = np.std(past_year['Close'])
    mean_1y = np.mean(past_year['Close'])
    
    return([n_,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,*quantiles_close_2y,*quantiles_close_1y])

In [ ]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
# = getStock(npa[0])
    
client = Client('192.168.3.100:8786')
client.restart()
future = client.map(getStock, npa)
results_prices = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_prices.append("error")
    else:
        results_prices.append(f.result()) 
client.close()


In [ ]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i]])

#getSector(npa[0])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getSector, npa)
results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_sectors.append("error")
    else:
        results_sectors.append(f.result()) 
client.close()


In [ ]:
len(np.unique([r[0] for r in results_sectors]))

In [ ]:
sectors_df = pd.DataFrame()

for r in range(0,len(results_sectors)):
    r_ = results_sectors[r]
    if(r_=='error'):
        pass#sectors_df = pd.concat([sectors_df,pd.DataFrame([np.nan],index=[i_],columns=['Sector'])],axis=0)
    else:
        s_ = np.array(r_)[1]
        i_ = np.array(r_)[0]
        sectors_df = pd.concat([sectors_df,pd.DataFrame([s_],index=[i_],columns=['Sector'])],axis=0)

missing_sectors = list(set(SP1500).difference([r[0] for r in results_sectors]))

for r in range(0,len(missing_sectors)):
    r_ = missing_sectors[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    sectors_df = pd.concat([sectors_df,pd.DataFrame(['error'],index=[r_],columns=['Sector'])],axis=0)

In [ ]:
#sectors_df

In [ ]:
#mktcap_df

In [ ]:
#np.sum(sectors_df.index.duplicated())

In [ ]:
#sectors_df.reindex(mktcap_df.index.duplicated())

In [ ]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i]])

#getMarketCap(npa[0])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getMarketCap, npa)
results_mktcp = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_mktcp.append("error")
    else:
        results_mktcp.append(f.result()) 
client.restart()
client.close()


In [ ]:
results_mktcp[0]

In [ ]:
#np.array([r[0] for r in results_mktcp])
#names_mktcap = [n[0] for n in results_mktcp]

mktcap_df = pd.DataFrame()

for r in range(0,len(results_mktcp)):
    r_ = results_mktcp[r]
    if(r_=='error'):
        pass#sectors_df = pd.concat([sectors_df,pd.DataFrame([np.nan],index=[i_],columns=['Sector'])],axis=0)
    else:
        s_ = np.array(r_)[1]
        #print(s_[0])
        i_ = np.array(r_)[0]
        mktcap_df = pd.concat([mktcap_df,pd.DataFrame([s_[0]],index=[i_],columns=['Market Cap'])],axis=0)

missing_mktcaps = list(set(SP1500).difference([n[0] for n in results_mktcp]))

for r in range(0,len(missing_mktcaps)):
    r_ = missing_mktcaps[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    mktcap_df = pd.concat([mktcap_df,pd.DataFrame([np.nan],index=[r_],columns=['Market Cap'])],axis=0)


In [ ]:
mktcap_df

In [ ]:
mktcap_df.sort_values(by='Market Cap',ascending=False)

In [ ]:
sectors_df.loc[~sectors_df.index.duplicated(), :].reindex(mktcap_df.loc[~mktcap_df.index.duplicated(), :].index)

In [ ]:
pd.concat([sectors_df,mktcap_df],axis=1)

In [ ]:
#total_mkt_cap = mktcap_df.replace([np.nan], 0).sum()
#mktcap_df['mkt_cap_pct'] = mktcap_df.replace([np.nan], 0)/total_mkt_cap

In [ ]:
#[rp[2] for rp in results_prices][0].info['sector']

In [ ]:

arr=np.array(names_prices)
pd.Series(arr).value_counts()
set_ = pd.Series(arr).value_counts()[pd.Series(arr).value_counts()>1]
if(len(set_)==0):
    duplicates = []
    pass
else:
    duplicates = set_.index[0]

In [ ]:
#len(results)

In [ ]:
names_prices = [rp[0] for rp in results_prices]

In [139]:
prices_df = pd.DataFrame()

#def process_df(data_):
    
for n in range(0,len(names_prices)):
    #print(names_prices[n])
    data_ = results_prices[n][1]
    data_['Symbol'] = names_prices[n]
    """
    if(np.sum(duplicates==names_prices[n])>0):
        #potential duplicate
        if((np.sum((np.unique(prices_df['Symbol']))==n))>0):
            pass
        else:
            prices_df = pd.concat([prices_df,data_],axis=0)            
    else:
    """
    prices_df = pd.concat([prices_df,data_],axis=0)

In [ ]:
#sectors_df.reindex(mktcap_df.index.duplicated())

In [148]:
npa = []
for s in names_prices:
    subset = prices_df.iloc[np.where(prices_df['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)
    

In [ ]:
"""
screenerMetrics = []

for n in npa:
    
    #data_df = pd.DataFrame(n[0])
    data_df = n
    n_ = data_df['Symbol'].values[-1]
    #print(n_)
    value = calculateMinMax(n)
    #print(value)
    screenerMetrics.append(value)
"""

In [169]:

screenerMetrics = []

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(calculateMinMax, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        screenerMetrics.append("error")
    else:
        screenerMetrics.append(f.result()) 
client.close()


In [152]:
#names_prices[np.where(np.array(names_prices)=='MMM')[0][0]]

'MMM'

53.459999084472656

In [161]:
#np.min(npa[np.where(np.array(names_prices)=='GBX')[0][0]]['Open'])

19.739999771118164

In [168]:
calculateMinMax(npa[np.where(np.array(names_prices)=='GBX')[0][0]])

['GBX',
 53.459999084472656,
 19.1200008392334,
 7.981057417427399,
 39.355337305674475,
 53.459999084472656,
 36.189998626708984,
 2.929218859465496,
 44.046614218884564,
 39.90999984741211,
 21.219999313354492,
 22.099000129699707,
 26.443500099182128,
 33.64750099182129,
 42.42499923706055,
 45.060001373291016,
 47.62730049133301,
 51.17900062561035,
 53.400001525878906,
 21.219999313354492,
 22.099000129699707,
 26.443500099182128,
 33.64750099182129,
 42.42499923706055,
 45.060001373291016,
 47.62730049133301,
 51.17900062561035,
 53.400001525878906]

In [172]:
writer = pd.ExcelWriter('SP1500_screener_multiple.xlsx', engine='xlsxwriter')


prices_df.to_excel(writer, sheet_name='SP1500Prices')
#prices_df.drop(['Sector', 'Market Cap'], axis=1).to_excel(writer, sheet_name='SP1500Prices')

sectors_df.to_excel(writer, sheet_name='stock_sectors')

mktcap_df.to_excel(writer, sheet_name='market_caps')

sp1500_index_df.to_excel(writer, sheet_name='SP1500')

pd.DataFrame(sp500).to_excel(writer, sheet_name='SP500')
pd.DataFrame(sp600).to_excel(writer, sheet_name='SP600')
pd.DataFrame(sp400).to_excel(writer, sheet_name='SP400')
pd.DataFrame(screenerMetrics,columns=['Symbol','max_2y','min_2y','std_2y','mean_2y','max_1y','min_1y','std_1y','mean_1y','latest_p','2yr_close_0pct','2yr_close_2pct','2yr_close_9pct','2yr_close_25pct','2yr_close_50pct','2yr_close_75pct','2yr_close_91pct','2yr_close_98pct','2yr_close_100pct','1yr_close_0pct','1yr_close_2pct','1yr_close_9pct','1yr_close_25pct','1yr_close_50pct','2yr_close_75pct','1yr_close_91pct','1yr_close_98pct','1yr_close_100pct']).to_excel(writer, sheet_name='Screener')
writer.save()

wb = openpyxl.load_workbook('SP1500_screener_multiple.xlsx')
#openpyxl.Workbook()
new_range = openpyxl.workbook.defined_name.DefinedName('Prices', attr_text='SP1500Prices!$A$1:$H$'+str(len(prices_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Sectors', attr_text='stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('market_caps', attr_text='market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SP1500', attr_text='SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))
wb.defined_names.append(new_range)

#wb.defined_names.append(new_range)
wb.save('SP1500_screener_multiple.xlsx')
wb.close()
#named_range = NamedRange("test_nr", [(new_sheet, "A1")])
#wb.add_named_range(new_range)
"""
workbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')
workbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))
workbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
workbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
workbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))

workbook.close()
"""

"\nworkbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')\nworkbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))\nworkbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))\nworkbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))\nworkbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))\n\nworkbook.close()\n"